# The Cross-Entropy Method

1. On CartPole

In [1]:
import gym
from collections import namedtuple
import numpy as np
from tensorboardX import SummaryWriter # to log values

import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# some hyperparams
HIDDEN_SIZE = 128
BATCH_SIZE = 16 # number of episodes to play on every batch
PERCENTILE = 70 # to limit good-result ones

In [3]:
class Net(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(Net, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions),
        )
        pass
    
    def forward(self, x):
        return self.net(x)

In [4]:
Episode = namedtuple("Episode", field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=
                         ['observation', 'action'])

# two classes to store the results

In [5]:
def iterate_batches(env, net, batch_size): # generate batch of data
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs= env.reset()
    sm = nn.Softmax(dim=1) # softmax function
    
    while True: # infinite dataset
        obs_v = torch.FloatTensor([obs])
        act_probs_v = sm(net(obs_v)) # map obs to action
        act_probs = act_probs_v.data.numpy()[0] # only one env at a time
        
        action= np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, _ = env.step(action) # step env randomly
        
        episode_reward += reward # gamma is 1
        step = EpisodeStep(observation=obs, action=action) # tuple, but named
        episode_steps.append(step)
        
        if is_done:
            e= Episode(reward= episode_reward, steps=episode_steps)
            batch.append(e)
            
            episode_reward= 0
            episode_steps= []
            next_obs = env.reset()
            
            if len(batch) == batch_size:
                yield batch # return it just now
                batch = []
                
        obs = next_obs

In [6]:
def filter_batch(batch, percentile):
    rewards = list( map(lambda s: s.reward, batch)) # episode.reward for every episode in batch
    reward_bound = np.percentile(rewards, percentile)
    reward_mean = float(np.mean(rewards)) # only for monitor
    
    # filter our episodes
    train_obs = []
    train_act= []
    for reward, steps in batch: # batch is filled with tuple
        if reward < reward_bound:
            continue # to simplify coding
        
        train_obs.extend([step.observation for step in steps])
        train_act.extend([step.action for step in steps])
        # not seperated by episodes
        
    train_obs_v = torch.FloatTensor(train_obs)
    train_act_v = torch.LongTensor(train_act)
    
    return train_obs_v, train_act_v, reward_bound, reward_mean

In [7]:
import pyvirtualdisplay, os
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()
# to enable recording

In [8]:
# main
env = gym.make("CartPole-v0")
env = gym.wrappers.Monitor(env,directory="cartpole_mon", force=True)

obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss() # instantize!!
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer= SummaryWriter(comment="-cartpole") # add comment

for iter_no, batch in enumerate(iterate_batches(
    env, net, BATCH_SIZE)): # infinite dataset using yeild
    obs_v, acts_v, reward_b, reward_m = filter_batch(
        batch, PERCENTILE)
    
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward() # backprop
    optimizer.step()
    
    print("%d: loss=%.3f, reward_mean=%.1f, reward_boud=%.1f" %(
          iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    
    if reward_m > 199.0:
        print("Solved!")
        break

writer.close()

0: loss=0.683, reward_mean=23.0, reward_boud=20.0
1: loss=0.667, reward_mean=23.2, reward_boud=27.0
2: loss=0.666, reward_mean=20.7, reward_boud=23.5
3: loss=0.665, reward_mean=24.0, reward_boud=27.0
4: loss=0.665, reward_mean=21.6, reward_boud=27.0
5: loss=0.662, reward_mean=34.9, reward_boud=41.5
6: loss=0.653, reward_mean=31.6, reward_boud=34.5
7: loss=0.638, reward_mean=42.9, reward_boud=54.5
8: loss=0.635, reward_mean=53.8, reward_boud=65.5
9: loss=0.617, reward_mean=55.2, reward_boud=69.0
10: loss=0.612, reward_mean=79.5, reward_boud=97.0
11: loss=0.610, reward_mean=67.9, reward_boud=85.5
12: loss=0.611, reward_mean=79.4, reward_boud=94.0
13: loss=0.605, reward_mean=82.9, reward_boud=106.5
14: loss=0.618, reward_mean=88.0, reward_boud=107.5
15: loss=0.586, reward_mean=77.1, reward_boud=94.5
16: loss=0.583, reward_mean=104.9, reward_boud=113.0
17: loss=0.575, reward_mean=108.1, reward_boud=139.0
18: loss=0.593, reward_mean=94.9, reward_boud=115.0
19: loss=0.569, reward_mean=99.5, 

2. On FrozenLake

In [9]:
e = gym.make("FrozenLake-v0")
print(e.observation_space)
print(e.action_space)
print(e.reset())
print(e.render())

Discrete(16)
Discrete(4)
0

SFFF
FHFH
FFFH
HFFG
None


In [10]:
# turn observation to one-hot vector
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env) # pass env!
        assert isinstance(env.observation_space, gym.spaces.Discrete)
        shape = (env.observation_space.n, ) # why tuple?
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32) # save blueprint
        pass
    
    def observation(self, observation): # remap previous observation
        res= np.copy(self.observation_space.low) # this returns [0,0,0,...0] ?
        res[observation] = 1.0 # fill it with number
        return res

In [11]:
# main -> exactly the same as cartpole!! beauty of rl
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0")) # wrap it
env = gym.wrappers.Monitor(env, directory="frozenlake-mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-frozenlake-naive")

for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
    obs_v, acts_v, reward_b, reward_m = filter_batch(batch, PERCENTILE)
    optimizer.zero_grad()
    action_scores_v = net(obs_v)
    loss_v = objective(action_scores_v, acts_v)
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, reward_mean=%.1f, reward_bound=%.1f" % (
        iter_no, loss_v.item(), reward_m, reward_b))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_bound", reward_b, iter_no)
    writer.add_scalar("reward_mean", reward_m, iter_no)
    if reward_m > 0.8:
        print("Solved!")
        break
writer.close()


0: loss=1.380, reward_mean=0.0, reward_bound=0.0
1: loss=1.361, reward_mean=0.0, reward_bound=0.0
2: loss=1.349, reward_mean=0.0, reward_bound=0.0
3: loss=1.302, reward_mean=0.0, reward_bound=0.0
4: loss=1.288, reward_mean=0.0, reward_bound=0.0
5: loss=1.244, reward_mean=0.0, reward_bound=0.0
6: loss=1.197, reward_mean=0.0, reward_bound=0.0
7: loss=1.152, reward_mean=0.0, reward_bound=0.0
8: loss=1.004, reward_mean=0.0, reward_bound=0.0
9: loss=1.164, reward_mean=0.0, reward_bound=0.0
10: loss=1.040, reward_mean=0.0, reward_bound=0.0
11: loss=0.938, reward_mean=0.0, reward_bound=0.0
12: loss=0.898, reward_mean=0.0, reward_bound=0.0
13: loss=0.814, reward_mean=0.0, reward_bound=0.0
14: loss=0.862, reward_mean=0.0, reward_bound=0.0
15: loss=0.788, reward_mean=0.1, reward_bound=0.0
16: loss=0.960, reward_mean=0.0, reward_bound=0.0
17: loss=0.779, reward_mean=0.0, reward_bound=0.0
18: loss=0.911, reward_mean=0.0, reward_bound=0.0
19: loss=0.741, reward_mean=0.0, reward_bound=0.0
20: loss=0

KeyboardInterrupt: 

#### does not improve over time!

3. tweaked Frozenlake

In [12]:
HIDDEN_SIZE = 128
BATCH_SIZE = 100 # increase batch size
PERCENTILE = 30 # more examples
GAMMA = 0.9 # discount rate to increase variability

In [13]:
def filter_batch(batch, percentile):
    filter_func = lambda s: s.reward * ( GAMMA ** len(s.steps)) # list including steps of episodes
    disc_rewards = list(map(filter_func, batch)) # returns list of rewards
    reward_bound = np.percentile(disc_rewards, percentile)
    
    train_obs= []
    train_act = []
    elite_batch= []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend([step.observation for step in example.steps])
            train_act.extend([step.action for step in example.steps])
            elite_batch.append(example)
            
    return elite_batch, train_obs, train_act, reward_bound

In [ ]:
# main
env = DiscreteOneHotWrapper(gym.make("FrozenLake-v0"))
env = gym.wrappers.Monitor(env, directory="frozenlake_tweaked_mon", force=True)
obs_size= env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
writer= SummaryWriter(comment="-frozenlake-tweaked")

full_batch = []

for iter_no, batch in enumerate(iterate_batches(
    env, net, BATCH_SIZE)):
    reward_mean = float(np.mean([s.reward for s in batch]))
    
    full_batch, obs, acts, reward_bound= filter_batch(full_batch + batch, PERCENTILE) # add full batch to our filter
    if not full_batch: # got nothing? # reuse full batch much like ema
        continue
    
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts) # put integer values
    full_batch= full_batch[-500:]
    optimizer.zero_grad()
    action_scores_v = net(obs_v) # get network's output 
    loss_v = objective(action_scores_v, acts_v) # and force it wide
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, rw_mean=%.3f, "
            "rw_bound=%.3f, batch=%d" % (
    iter_no, loss_v.item(), reward_mean,
    reward_bound, len(full_batch)))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_mean", reward_mean, iter_no)
    writer.add_scalar("reward_bound", reward_bound, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break
writer.close()

0: loss=1.394, rw_mean=0.010, rw_bound=0.000, batch=1
1: loss=1.379, rw_mean=0.010, rw_bound=0.000, batch=2
2: loss=1.372, rw_mean=0.000, rw_bound=0.000, batch=2
3: loss=1.340, rw_mean=0.030, rw_bound=0.000, batch=5
4: loss=1.335, rw_mean=0.000, rw_bound=0.000, batch=5
5: loss=1.349, rw_mean=0.040, rw_bound=0.000, batch=9
6: loss=1.333, rw_mean=0.040, rw_bound=0.000, batch=13
7: loss=1.328, rw_mean=0.020, rw_bound=0.000, batch=15
8: loss=1.324, rw_mean=0.000, rw_bound=0.000, batch=15
9: loss=1.325, rw_mean=0.020, rw_bound=0.000, batch=17
10: loss=1.323, rw_mean=0.020, rw_bound=0.000, batch=19
11: loss=1.333, rw_mean=0.030, rw_bound=0.000, batch=22
12: loss=1.337, rw_mean=0.030, rw_bound=0.000, batch=25
13: loss=1.336, rw_mean=0.010, rw_bound=0.000, batch=26
14: loss=1.334, rw_mean=0.000, rw_bound=0.000, batch=26
15: loss=1.333, rw_mean=0.010, rw_bound=0.000, batch=27
16: loss=1.331, rw_mean=0.010, rw_bound=0.000, batch=28
17: loss=1.324, rw_mean=0.040, rw_bound=0.000, batch=32
18: loss

In [15]:
# nonslippery
# main
env = gym.envs.toy_text.frozen_lake.FrozenLakeEnv(is_slippery=False)
env.spec= gym.spec("FrozenLake-v0")
env = gym.wrappers.TimeLimit(env, max_episode_steps=100)


env = DiscreteOneHotWrapper(env)
env = gym.wrappers.Monitor(env, directory="frozenlake_tweaked_nonslip_mon", force=True)
obs_size= env.observation_space.shape[0]
n_actions = env.action_space.n

net = Net(obs_size, HIDDEN_SIZE, n_actions)
objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.001)
writer= SummaryWriter(comment="-frozenlake-tweaked-nonslip")

full_batch = []

for iter_no, batch in enumerate(iterate_batches(
    env, net, BATCH_SIZE)):
    reward_mean = float(np.mean([s.reward for s in batch]))
    
    full_batch, obs, acts, reward_bound= filter_batch(full_batch + batch, PERCENTILE) # add full batch to our filter
    if not full_batch: # got nothing? # reuse full batch much like ema
        continue
    
    obs_v = torch.FloatTensor(obs)
    acts_v = torch.LongTensor(acts) # put integer values
    full_batch= full_batch[-500:]
    optimizer.zero_grad()
    action_scores_v = net(obs_v) # get network's output 
    loss_v = objective(action_scores_v, acts_v) # and force it wide
    loss_v.backward()
    optimizer.step()
    print("%d: loss=%.3f, rw_mean=%.3f, "
            "rw_bound=%.3f, batch=%d" % (
    iter_no, loss_v.item(), reward_mean,
    reward_bound, len(full_batch)))
    writer.add_scalar("loss", loss_v.item(), iter_no)
    writer.add_scalar("reward_mean", reward_mean, iter_no)
    writer.add_scalar("reward_bound", reward_bound, iter_no)
    if reward_mean > 0.8:
        print("Solved!")
        break
writer.close()

0: loss=1.360, rw_mean=0.010, rw_bound=0.000, batch=1
1: loss=1.353, rw_mean=0.000, rw_bound=0.000, batch=1
2: loss=1.346, rw_mean=0.000, rw_bound=0.000, batch=1
3: loss=1.362, rw_mean=0.010, rw_bound=0.000, batch=2
4: loss=1.351, rw_mean=0.010, rw_bound=0.000, batch=3
5: loss=1.355, rw_mean=0.010, rw_bound=0.000, batch=4
6: loss=1.355, rw_mean=0.010, rw_bound=0.000, batch=5
7: loss=1.351, rw_mean=0.000, rw_bound=0.000, batch=5
8: loss=1.351, rw_mean=0.030, rw_bound=0.000, batch=8
9: loss=1.345, rw_mean=0.040, rw_bound=0.000, batch=12
10: loss=1.339, rw_mean=0.010, rw_bound=0.000, batch=13
11: loss=1.335, rw_mean=0.000, rw_bound=0.000, batch=13
12: loss=1.330, rw_mean=0.000, rw_bound=0.000, batch=13
13: loss=1.332, rw_mean=0.010, rw_bound=0.000, batch=14
14: loss=1.332, rw_mean=0.010, rw_bound=0.000, batch=15
15: loss=1.336, rw_mean=0.030, rw_bound=0.000, batch=18
16: loss=1.331, rw_mean=0.040, rw_bound=0.000, batch=22
17: loss=1.326, rw_mean=0.010, rw_bound=0.000, batch=23
18: loss=1.